<h2>Segmenting and Clustering Neighborhoods in Toronto-3<h2>

<h4>Explore and cluster the neighborhoods in Toronto.
Generate maps to visualize your neighborhoods and how they cluster together<h4>

In [8]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


In [9]:

List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [10]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)


In [12]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [13]:
df=df[df['Borough']!='Not assigned']

In [ ]:
df[df['Neighborhood']=='Not assigned']=df['Borough']

In [14]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:

temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

df_merge = pd.merge(df, temp_df, on='Postalcode')

df_merge.drop(['Neighborhood'],axis=1,inplace=True)

df_merge.drop_duplicates(inplace=True)

df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
8,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [17]:

df_merge.shape

(103, 3)

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,43.770992,-79.216917,Scarborough,"Woburn, Woburn"
4,M1H,43.773136,-79.239476,Scarborough,"Cedarbrae, Cedarbrae"


In [19]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [20]:

toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
43,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
44,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790


In [31]:

CLIENT_ID = 'TMNVXVHFEN15FKHI03NIQ3BCGH3IPABX5ZIKQ2B4CEKLSLSS' 
CLIENT_SECRET = 'UMJM513Y5OKR1PW3XU1AHNXQG4XZ1KUTB2YAUIZWXOVN1SUS'
VERSION = '20180604'

In [34]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
             v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches, The Beaches
The Danforth West, Riverdale, The Danforth West, Riverdale
India Bazaar, The Beaches West, India Bazaar, The Beaches West
Studio District, Studio District
Lawrence Park, Lawrence Park
Davisville North, Davisville North
North Toronto West,  Lawrence Park, North Toronto West,  Lawrence Park
Davisville, Davisville
Moore Park, Summerhill East, Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park, Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale, Rosedale
St. James Town, Cabbagetown, St. James Town, Cabbagetown
Church and Wellesley, Church and Wellesley
Regent Park, Harbourfront, Regent Park, Harbourfront
Garden District, Ryerson, Garden District, Ryerson
St. James Town, St. James Town
Berczy Park, Berczy Park
Central Bay Street, Central Bay Street
Richmond, Adelaide, King, Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands, Harbourfront East, Union Station, Toronto Islands
To

In [36]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Beaches, The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"The Beaches, The Beaches",43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,"The Beaches, The Beaches",43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,"The Beaches, The Beaches",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Beaches, The Beaches",43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [37]:

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Berczy Park, Berczy Park",58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place, Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport, CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
"Central Bay Street, Central Bay Street",63,63,63,63,63,63
"Christie, Christie",16,16,16,16,16,16
"Church and Wellesley, Church and Wellesley",80,80,80,80,80,80
"Commerce Court, Victoria Hotel, Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Davisville North, Davisville North",8,8,8,8,8,8


In [38]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1604, 228)

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Berczy Park, Berczy Park",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place, ...",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,"Central Bay Street, Central Bay Street",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.015873,0.015873


In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Berczy Park, Berczy Park",Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Beer Bar,Café
1,"Brockton, Parkdale Village, Exhibition Place, ...",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
4,"Central Bay Street, Central Bay Street",Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Yoga Studio,Juice Bar


In [42]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [43]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio
41,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Pub,Pizza Place,Lounge
42,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572,1,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Ice Cream Shop,Steakhouse,Intersection,Fish & Chips Shop,Brewery,Italian Restaurant,Food & Drink Shop
43,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923,1,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Clothing Store
44,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790,4,Park,Bus Line,Swim School,Business Service,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


In [44]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Berczy Park, Berczy Park",Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Beer Bar,Café
1,1,"Brockton, Parkdale Village, Exhibition Place, ...",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,1,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Light Rail Station
3,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
4,1,"Central Bay Street, Central Bay Street",Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Yoga Studio,Juice Bar


In [45]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [47]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters